<a href="https://colab.research.google.com/github/doomba32/python/blob/main/%EA%B2%BD%EA%B8%B0%EB%8F%84%EB%A7%9B%EC%A7%91%ED%98%84%ED%99%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#필요한 파이썬 모듈을 import
from google.colab import files #google colab에 파일 업로드 위한 모듈
import io #업로드된 파일을 읽기위한 모듈
import pandas as pd #데이터 분석을 위한 모듈

In [2]:
#이 코드 실행 후, 아래에서 '파일 선택' 하여 '맛집현황.csv' 업로드
uploaded = files.upload()

Saving 맛집현황.csv to 맛집현황.csv


In [1]:
# 업로드된 '맛집현황.csv' 파일을 '맛집'이라는 변수로 업로드 (주의: 해당 파일은 euc-kr로 encoding되어 있어 이를 지정해줘야 정상적으로 읽을 수 있음)
맛집 = pd.read_csv(io.BytesIO(uploaded['맛집현황.csv']),encoding='euc-kr')

In [8]:
# 해당 파일을 미리 받지 못한경우, 아래 주석처리된 코드를 대신 사용
#맛집 = pd.read_csv('http://doomba32.github.io/python/data/Restaurant.csv', encoding='euc-kr')

In [7]:
#앞쪽 데이터 확인
맛집.head()

,시군명,음식점명,맛집전화번호,대표음식명,소재지우편번호,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도
0,가평군,가평축협 한우명가,031-581-1592,푸른연잎한우명품꽃등심,12422,경기도 가평군 가평읍 달전로 19,경기도 가평군 가평읍 달전리 382-1번지,37.815844,127.516128
1,고양시,청정바지락칼국수,031-912-7676,천년초들깨수제비,10359,경기도 고양시 일산동구 일산로463번길 7,경기도 고양시 일산동구 정발산동 1148번지,37.673707,126.775375
2,고양시,양촌리아구,031-911-0430,아구탕,10218,경기도 고양시 일산서구 대화2로 152,경기도 고양시 일산서구 대화동 762-3번지,37.671931,126.736219
3,고양시,정통중화요리 남궁,031-911-3702,"해물고추짬뽕, 양장피잡채",10367,경기도 고양시 일산서구 일산로 682,경기도 고양시 일산서구 대화동 2101번지,37.682042,126.753550
4,고양시,야구장농원,031-964-2884,오리진흙구이,10313,경기도 고양시 일산동구 견달산로 351,경기도 고양시 일산동구 문봉동 102-1번지,37.697102,126.819819


In [8]:
#정보 출력
맛집.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군명       137 non-null    object 
 1   음식점명      137 non-null    object 
 2   맛집전화번호    137 non-null    object 
 3   대표음식명     137 non-null    object 
 4   소재지우편번호   137 non-null    int64  
 5   소재지도로명주소  137 non-null    object 
 6   소재지지번주소   137 non-null    object 
 7   WGS84위도   137 non-null    float64
 8   WGS84경도   137 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 9.8+ KB


In [10]:
#일부 column 명 변경
맛집.rename(columns={맛집.columns[7] : '위도'}, inplace=True)
맛집.rename(columns={맛집.columns[8] : '경도'}, inplace=True)
맛집.head()

,시군명,음식점명,맛집전화번호,대표음식명,소재지우편번호,소재지도로명주소,소재지지번주소,위도,경도
0,가평군,가평축협 한우명가,031-581-1592,푸른연잎한우명품꽃등심,12422,경기도 가평군 가평읍 달전로 19,경기도 가평군 가평읍 달전리 382-1번지,37.815844,127.516128
1,고양시,청정바지락칼국수,031-912-7676,천년초들깨수제비,10359,경기도 고양시 일산동구 일산로463번길 7,경기도 고양시 일산동구 정발산동 1148번지,37.673707,126.775375
2,고양시,양촌리아구,031-911-0430,아구탕,10218,경기도 고양시 일산서구 대화2로 152,경기도 고양시 일산서구 대화동 762-3번지,37.671931,126.736219
3,고양시,정통중화요리 남궁,031-911-3702,"해물고추짬뽕, 양장피잡채",10367,경기도 고양시 일산서구 일산로 682,경기도 고양시 일산서구 대화동 2101번지,37.682042,126.753550
4,고양시,야구장농원,031-964-2884,오리진흙구이,10313,경기도 고양시 일산동구 견달산로 351,경기도 고양시 일산동구 문봉동 102-1번지,37.697102,126.819819


In [4]:
#매탄고등학교의 위도, 경로 입력
# https://pilot.duni.io/util/ 에서 "경기도 수원시 영통구 매탄로126번길 11" 주소를 좌표로 변환
위도 = 37.262576051
경도 = 127.04475557

In [2]:

# 거리 계산을 위한 harversine library 설치
!pip install haversine

In [13]:
#haversine(( 37.262576051,127.04475557),(37.815844	,127.516128))
haversine(( 37.262576051,127.04475557),(37.258157	,127.181793))

12.137669756366737

In [31]:
#거리계산  함수 정의
# math 모듈의 pi, acos, cos, sin 사용하여 두 지점의 거리(km 단위) 계산

# 참고1  :http://www.mapanet.eu/EN/Resources/Script-Distance.htm
# 참고2 : https://blog.naver.com/PostView.naver?blogId=loyz&logNo=222379027185

from math import *

def 거리계산(위도1,경도1,위도2,경도2):
  위도1 = 위도1*pi /180
  경도1 = 경도1*.pi /180
  위도2 = 위도2*pi /180
  경도2 = 경도2*pi /180
  return 6378.137*acos(cos(위도1)cos(위도2)*cos(경도2-경도1)+sin(위도1)*sin(위도2))


In [33]:
거리계산(위도1,경도1,위도2,경도2)

1.1625182300699448

In [29]:
#매탄고등학교로부터 각 맛집으로의 거리 계산
#데이터 상의 각 맛집의 '위도', '경도' 를 거리계산 함수의 인자로 넣어줌
# apply 함수 사용시 column 방향으로 참조하므로 axis=1

맛집['거리'] = 맛집.apply(lambda x:거리계산(위도 ,경도, x['위도'] ,x['경도']), axis=1)

In [35]:

#매탄고등학교로부터 가장 가까운 5개의 맛집 표시
맛집.sort_values(by='거리', ascending=True).head(5)

,시군명,음식점명,맛집전화번호,대표음식명,소재지우편번호,소재지도로명주소,소재지지번주소,위도,경도,거리
50,수원시,한국관,031-217-8889,수원양념왕갈비,16491,경기도 수원시 팔달구 권선로741번길 16-8,경기도 수원시 팔달구 인계동 1134-1번지,37.258157,127.032867,1.162517
55,수원시,신라갈비,031-212-2354,생갈비,16518,경기도 수원시 영통구 동수원로 538,경기도 수원시 영통구 원천동 89-7번지,37.273713,127.049199,1.300723
48,수원시,남촌,031-239-0411,홍어삼합,16571,경기도 수원시 권선구 권광로123번길 15,경기도 수원시 권선구 권선동 1022번지,37.258879,127.029700,1.395979
56,수원시,본수원갈비,031-211-5952,생갈비,16496,경기도 수원시 팔달구 중부대로223번길 41,경기도 수원시 팔달구 우만동 51-20번지,37.278041,127.040441,1.763442
52,수원시,가보정,031-238-3883,양념갈비,16481,경기도 수원시 팔달구 장다리로 282,경기도 수원시 팔달구 인계동 958-1번지,37.274124,127.028668,1.919341


In [34]:
#대표음식명에 '갈비'를 포함하는 5개의 맛집 표시
맛집[맛집['대표음식명'].str.contains('갈비')].head(5)

,시군명,음식점명,맛집전화번호,대표음식명,소재지우편번호,소재지도로명주소,소재지지번주소,위도,경도,거리
7,광주시,능골한우가,031-797-0255,"갈비탕, 한우",12773,경기도 광주시 오포읍 오포로 147,경기도 광주시 오포읍 능평리 146-6번지,37.345677,127.181793,15.258307
15,광주시,광주본갈비,031-769-6363,돼지갈비,12741,경기도 광주시 경안천로 33,경기도 광주시 송정동 94-1번지,37.414001,127.264065,25.708009
27,동두천시,동동갈비,031-864-3093,"돼지갈비, 갈비탕",11330,경기도 동두천시 어수로101번길 13,경기도 동두천시 생연동 670-3번지,37.902802,127.053672,71.273945
30,부천시,참맛진숯불갈비,032-325-6464,한우암소갈비살,14546,경기도 부천시 석천로177번길 16,경기도 부천시 중동 1163-11번지,37.503423,126.761971,36.667341
47,수원시,명가정,031-255-9916,소양념갈비,16603,경기도 수원시 권선구 탑동로57번길 29,경기도 수원시 권선구 탑동 746-2번지,37.273204,126.972260,6.530448


In [36]:
#대표음식명에 '갈비'를 포함하는 맛집 중 매탄고등학교로부터 가장 가까운 5개의 맛집 표시
맛집[맛집['대표음식명'].str.contains('갈비')].sort_values(by='거리', ascending=True).head(5)

,시군명,음식점명,맛집전화번호,대표음식명,소재지우편번호,소재지도로명주소,소재지지번주소,위도,경도,거리
50,수원시,한국관,031-217-8889,수원양념왕갈비,16491,경기도 수원시 팔달구 권선로741번길 16-8,경기도 수원시 팔달구 인계동 1134-1번지,37.258157,127.032867,1.162517
55,수원시,신라갈비,031-212-2354,생갈비,16518,경기도 수원시 영통구 동수원로 538,경기도 수원시 영통구 원천동 89-7번지,37.273713,127.049199,1.300723
56,수원시,본수원갈비,031-211-5952,생갈비,16496,경기도 수원시 팔달구 중부대로223번길 41,경기도 수원시 팔달구 우만동 51-20번지,37.278041,127.040441,1.763442
52,수원시,가보정,031-238-3883,양념갈비,16481,경기도 수원시 팔달구 장다리로 282,경기도 수원시 팔달구 인계동 958-1번지,37.274124,127.028668,1.919341
54,수원시,송풍가든,031-252-4700,양념갈비,16300,경기도 수원시 장안구 경수대로 1013,경기도 수원시 장안구 송죽동 440-3번지,37.306177,127.000649,6.230467


In [37]:
#우리집의 위도,경도 입력
# https://pilot.duni.io/util/ 에서 주소를 좌표로 변환
위도=37.224512955
경도=127.05938805

In [44]:
#우리집로부터 각 맛집으로의 거리 계산
#데이터 상의 각 맛집의 '위도', '경도' 를 거리계산 함수의 인자로 넣어줌
# apply 함수 사용시 column 방향으로 참조하므로 axis=1

맛집['우리집거리'] = 맛집.apply(lambda x:거리계산(위도 ,경도, x['위도'] ,x['경도']), axis=1)

In [45]:
#(매탄고로부터의 거리) + (우리집로부터의 거리) 가 가장 짧은  5개의 맛집 표시

맛집['거리합'] = 맛집['거리'] + 맛집['우리집거리']

맛집.sort_values(by='거리합', ascending=True).head(5)

,시군명,음식점명,맛집전화번호,대표음식명,소재지우편번호,소재지도로명주소,소재지지번주소,위도,경도,거리,우리집거리,거리합,우리집까지거리
49,수원시,서윤한정식,031-273-2345,"행복 한정식, 서윤특선",16679,경기도 수원시 영통구 덕영대로1499번길 28,경기도 수원시 영통구 영통동 977-1번지,37.246850,127.055663,1.999660,2.508398,4.508057,2.508398
50,수원시,한국관,031-217-8889,수원양념왕갈비,16491,경기도 수원시 팔달구 권선로741번길 16-8,경기도 수원시 팔달구 인계동 1134-1번지,37.258157,127.032867,1.162517,4.421651,5.584168,4.421651
48,수원시,남촌,031-239-0411,홍어삼합,16571,경기도 수원시 권선구 권광로123번길 15,경기도 수원시 권선구 권선동 1022번지,37.258879,127.029700,1.395979,4.642982,6.038961,4.642982
55,수원시,신라갈비,031-212-2354,생갈비,16518,경기도 수원시 영통구 동수원로 538,경기도 수원시 영통구 원천동 89-7번지,37.273713,127.049199,1.300723,5.550809,6.851532,5.550809
56,수원시,본수원갈비,031-211-5952,생갈비,16496,경기도 수원시 팔달구 중부대로223번길 41,경기도 수원시 팔달구 우만동 51-20번지,37.278041,127.040441,1.763442,6.190687,7.954129,6.190687
